# Data Wrangling
* [1 Data Collection](#1_Data_Collection)
    * [1.1 Importing Taxi Trip Datasets](#1.1_Importing_Taxi_Trip_Datasets)
    * [1.2 Concatinating Into One DataFrame](#1.2_Concatinating_Into_One_DataFrame)
    * [1.3 Importing Additional Datasets](#1.3_Importing_Additional_Datasets)
    * [1.4 Data Merging](#1.4_Data_Merging)
    * [1.5 Applying Merge On Zones And Population](#1.5_Applying_Merge_On_All_Datasets)
    * [1.6 Removing Whitespaces](#1.6_Removing_whitespaces)
    * [1.7_Merging_on_boroughs](#1.7_Merging_on_boroughs)
    * [1.8 Merging on Taxi Data](#1.8_Merging_On_Taxi_Data)
* [2 Data Definition](#2_Data_Definition)
    * [2.1 Data Columns](#2.1_Data_Columns)
    * [2.2 Data Types](#2.2_Data_Types)
    * [2.3 Descriptions_Of_Columns](#2.3_Descriptions_Of_Columns)
    * [2.4 Counts_And_Percents](#2.4_Counts_And_Percents)
* [3 Data Cleaning](#3_Data_Cleaning)
    * [3.1 Filling NULL_Values](#3.1_Filling_NULL_values)
    * [3.2_Checking_For_Duplicates](#3.2_Checking_For_Duplicates)
    * [3.3_Finished_Cleaning](#3.3_Finished_Cleaning)

# 1 Data Collection<a id='1_Data_Collection'></a>

In [40]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime

In [2]:
pd.set_option('display.max_columns', 500)

# 1.1  Importing Taxi Trip Datasets <a id='1.1_Importing_Taxi_Trip_Datasets'></a>
The data is stored in a dataset folder in my directory and the data for each month is stored in a separate file. I will be importing each file in its own dataframe and then concatinating them into one big dataframe.

In [3]:
# The data is stored in a dataset folder in my directory.
# The data for each month is stored in a separate file.
january_data = pd.read_csv('./datasets/yellow_tripdata_2020-01.csv')
february_data = pd.read_csv('./datasets/yellow_tripdata_2020-02.csv')
march_data = pd.read_csv('./datasets/yellow_tripdata_2020-03.csv')
april_data = pd.read_csv('./datasets/yellow_tripdata_2020-04.csv')
may_data = pd.read_csv('./datasets/yellow_tripdata_2020-05.csv')
june_data = pd.read_csv('./datasets/yellow_tripdata_2020-06.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 1.2 Concatinating Into One DataFrame <a id='1.2_Concatinating_Into_One_DataFrame'></a>
The following cell will combine the entire taxi data into one big dataframe by concating all the monthly data that was imported/

In [4]:
pandemic_taxi_data = january_data
pandemic_taxi_data = pandemic_taxi_data.append([february_data, march_data, april_data,
                                                may_data, june_data]).reset_index(drop=True)
pandemic_taxi_data.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
16847773,NaN,2020-06-30 23:05:00,2020-06-30 23:32:00,NaN,12.96,NaN,NaN,17,69,NaN,32.91,0.0,0.5,2.75,6.12,0.3,42.58,0.0
16847774,NaN,2020-06-30 23:21:47,2020-06-30 23:25:24,NaN,0.36,NaN,NaN,41,41,NaN,11.45,0.0,0.5,2.75,0.00,0.3,15.00,0.0
16847775,NaN,2020-06-30 23:34:00,2020-06-30 23:44:00,NaN,2.36,NaN,NaN,242,81,NaN,18.45,0.0,0.5,2.75,0.00,0.3,22.00,0.0
16847776,NaN,2020-06-30 23:22:47,2020-06-30 23:42:01,NaN,5.50,NaN,NaN,14,118,NaN,15.90,0.0,0.5,6.23,12.24,0.3,35.17,0.0
16847777,NaN,2020-06-30 23:56:18,2020-07-01 00:27:19,NaN,9.59,NaN,NaN,61,137,NaN,29.68,0.0,0.5,0.00,0.00,0.3,32.98,2.5


# 1.3 Importing Additional Datasets<a id='1.3_Importing_Additional_Datasets'></a>

The next two datasets are the taxi zones, which holds data that lets us know where a LocationID is located in the city and a population dataset that lets us know how many people are in each borough.

In [5]:
taxi_zones = pd.read_csv('./datasets/taxi_zones.csv')
taxi_zones.head(10)

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens
2,3,0.084341,MULTIPOLYGON (((-73.84792614099985 40.87134223...,0.000314,Allerton/Pelham Gardens,3,Bronx
3,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan
4,5,0.092146,MULTIPOLYGON (((-74.17421738099989 40.56256808...,0.000498,Arden Heights,5,Staten Island
5,6,0.150491,MULTIPOLYGON (((-74.06367318899999 40.60219816...,0.000606,Arrochar/Fort Wadsworth,6,Staten Island
6,7,0.107417,MULTIPOLYGON (((-73.90413637799996 40.76752031...,0.000390,Astoria,7,Queens
7,8,0.027591,MULTIPOLYGON (((-73.92334041500001 40.77512891...,0.000027,Astoria Park,8,Queens
8,9,0.099784,MULTIPOLYGON (((-73.78502434699996 40.76103651...,0.000338,Auburndale,9,Queens
9,24,0.047000,MULTIPOLYGON (((-73.95953658899997 40.79871852...,0.000061,Bloomingdale,24,Manhattan


In [6]:
borough_population = pd.read_csv('./datasets/2020_borough_population.csv')
borough_population

,Age Group,Borough,2010,2020,Change in Percent - 2010-2020
0,Total,New York City,8242624,8550971,3.74
1,Total,Bronx,1385108,1446788,4.45
2,Total,Brooklyn,2552911,2648452,3.74
3,Total,Manhattan,1585873,1638281,3.30
4,Total,Queens,2250002,2330295,3.57
5,Total,Staten Island,468730,487155,3.93
6,School-Age (5 to17),New York City,1260400,1282814,1.78
7,School-Age (5 to17),Bronx,265052,259013,-2.28
8,School-Age (5 to17),Brooklyn,424704,441049,3.85
9,School-Age (5 to17),Manhattan,157856,162931,3.21


# 1.4. Data Merging <a id='1.4_Data_Merging'></a>

# 1.5 Applying Merge On All Datasets <a id='1.5_Applying_Merge_On_All_Datasets'></a>
The next cells will focus on merging taxi_zones and borough_population based off their boroughs. The idea is to have the columns merge and have a population entry for each borough entry in taxi_zones.

In [7]:
borough_population['Borough'].unique()

array(['New York City', '   Bronx', '   Brooklyn', '   Manhattan',
       '   Queens', '   Staten Island'], dtype=object)

In [8]:
taxi_zones['borough'].unique()

array(['EWR', 'Queens', 'Bronx', 'Manhattan', 'Staten Island', 'Brooklyn'],
      dtype=object)

# 1.6 Removing Whitespaces <a id = '1.6_Removing_whitespaces'></a>
It seems that they have the merging conditions we want but the values of borough_population has their row values with extra whitespace. Data cleaning will need to be used before merging them

In [9]:
# In order to elimimate whitespaces, we must convert the values in the 'Borough' column to strings,
# then we can strip the whitespaces out.
borough_population['Borough'] = borough_population['Borough'].str.strip()
# The borough population data needs to remove the values that do not have the total population of a borough
borough_population = borough_population.loc[borough_population['Age Group'] == 'Total']
borough_population['Borough'].unique()

array(['New York City', 'Bronx', 'Brooklyn', 'Manhattan', 'Queens',
       'Staten Island'], dtype=object)

# 1.7 Merging on boroughs <a id='1.7_Merging_on_boroughs'></a>
The next step is to merge taxi_zones and borough_population on their 'borough's. The only columns that will be needed from the taxis data is 'Borough' and '2020'.

In [10]:
# Next step, we merge. We will merge them based off their 'borough' columns.
# Keeping the keys in order of the taxi_zones.
taxi_zones = taxi_zones.merge(borough_population[['Borough','2020']], left_on='borough',
                              right_on='Borough', how='left')
taxi_zones.head()

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough,Borough,2020
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR,NaN,NaN
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens,Queens,2330295.0
2,3,0.084341,MULTIPOLYGON (((-73.84792614099985 40.87134223...,0.000314,Allerton/Pelham Gardens,3,Bronx,Bronx,1446788.0
3,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan,Manhattan,1638281.0
4,5,0.092146,MULTIPOLYGON (((-74.17421738099989 40.56256808...,0.000498,Arden Heights,5,Staten Island,Staten Island,487155.0


# 1.8 Merging on Taxi ID <a id='1.8_Merging_On_Taxi_Data'></a>

The final merges consists of bringing the LocationID, zones and Borough into the taxi data table.

In [11]:
# The next step is to merge the taxi ride data into the taxi zones
# To avoid 
pandemic_taxi_data = pandemic_taxi_data.merge(taxi_zones[['zone', 'LocationID', 'Borough']],
                                              left_on=['PULocationID'],
                                              right_on=['LocationID'])
pandemic_taxi_data = pandemic_taxi_data.rename(columns={'zone': 'zone_PU',
                                  'LocationID': 'LocationID_PU',
                                  'Borough': 'Borough_PU'})

In [12]:
pandemic_taxi_data = pandemic_taxi_data.merge(taxi_zones[['zone', 'LocationID', 'Borough']],
                                             left_on=['DOLocationID'],
                                             right_on=['LocationID'])
pandemic_taxi_data = pandemic_taxi_data.rename(columns={'zone': 'zone_DO',
                                                       'LocationID': 'LocationID_DO',
                                                       'Borough': 'Borough_DO'})
pandemic_taxi_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,zone_PU,LocationID_PU,Borough_PU,zone_DO,LocationID_DO,Borough_DO
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan
1,2.0,2020-01-01 00:24:22,2020-01-01 00:25:54,1.0,0.56,1.0,N,238,239,1.0,3.5,0.5,0.5,1.70,0.0,0.3,9.00,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan
2,2.0,2020-01-01 00:46:46,2020-01-01 00:49:17,1.0,0.62,1.0,N,238,239,1.0,4.5,0.5,0.5,1.00,0.0,0.3,9.30,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan
3,1.0,2020-01-01 00:17:53,2020-01-01 00:22:47,1.0,0.80,1.0,Y,238,239,1.0,5.5,3.0,0.5,1.85,0.0,0.3,11.15,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan
4,2.0,2020-01-01 00:49:45,2020-01-01 00:54:12,1.0,0.86,1.0,N,238,239,1.0,5.5,0.5,0.5,1.86,0.0,0.3,11.16,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan


# 1.9 Creating Dummy Variblaes <a id= '1.9Creating_Dummy_Variables'></a>

This will create dummy variables for the Borough_PU and Borough_DP.

In [13]:
VendorID = pandemic_taxi_data['VendorID']
RateCodeID = pandemic_taxi_data['RatecodeID']
Payment_Type = pandemic_taxi_data['payment_type']
Borough_PU = pandemic_taxi_data['Borough_PU']
Borough_DO = pandemic_taxi_data['Borough_DO']

VendorID_dummies = pd.get_dummies(VendorID)
RateCodeID = pd.get_dummies(RateCodeID)
Payment_Type = pd.get_dummies(Payment_Type)
PU_dummies = pd.get_dummies(Borough_PU)
DO_dummies = pd.get_dummies(Borough_DO)

In [14]:
RateCodeID.columns = ['RateCode_1', 'RateCode_2', 'RateCode_3',
                       'RateCode_4', 'RateCode_5', 'RateCode_6', 'RateCode_99']
VendorID_dummies.columns = ['VendorID_1', 'VendorID_2']
Payment_Type.columns = ['payment_type_1', 'payment_type_2', 'payment_type_3', 'payment_type_4', 'payment_type_5']
PU_dummies.columns = ['Bronx_PU', 'Brooklyn_PU', 'Manhattan_PU', 'Queens_PU', 'Staten_Island_PU']
DO_dummies.columns = ['Bronx_DO', 'Brooklyn_DO', 'Manhattan_DO', 'Queens_DO', 'Staten_Island_DO']

In [15]:
pandemic_taxi_data = pandemic_taxi_data.join(VendorID_dummies)
pandemic_taxi_data = pandemic_taxi_data.join(RateCodeID)
pandemic_taxi_data = pandemic_taxi_data.join(Payment_Type)
pandemic_taxi_data = pandemic_taxi_data.join(PU_dummies)
pandemic_taxi_data = pandemic_taxi_data.join(DO_dummies)
pandemic_taxi_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,zone_PU,LocationID_PU,Borough_PU,zone_DO,LocationID_DO,Borough_DO,VendorID_1,VendorID_2,RateCode_1,RateCode_2,RateCode_3,RateCode_4,RateCode_5,RateCode_6,RateCode_99,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,Bronx_PU,Brooklyn_PU,Manhattan_PU,Queens_PU,Staten_Island_PU,Bronx_DO,Brooklyn_DO,Manhattan_DO,Queens_DO,Staten_Island_DO
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1,2.0,2020-01-01 00:24:22,2020-01-01 00:25:54,1.0,0.56,1.0,N,238,239,1.0,3.5,0.5,0.5,1.70,0.0,0.3,9.00,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0
2,2.0,2020-01-01 00:46:46,2020-01-01 00:49:17,1.0,0.62,1.0,N,238,239,1.0,4.5,0.5,0.5,1.00,0.0,0.3,9.30,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0
3,1.0,2020-01-01 00:17:53,2020-01-01 00:22:47,1.0,0.80,1.0,Y,238,239,1.0,5.5,3.0,0.5,1.85,0.0,0.3,11.15,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4,2.0,2020-01-01 00:49:45,2020-01-01 00:54:12,1.0,0.86,1.0,N,238,239,1.0,5.5,0.5,0.5,1.86,0.0,0.3,11.16,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0


# 2. Data Definition <a id='2_Data_Definition'></a>

The following cells will begin to explore what the data holds and what the data looks like.

# 2.1 Data Columns <a id='2.1_Data_Columns'></a>

The names of the Data Columns that are present in the data table.

In [16]:
len(pandemic_taxi_data.columns)

48

In [17]:
for i in range(len(pandemic_taxi_data.columns)):
    print('{}. {}'.format(i + 1, pandemic_taxi_data.columns[i]))

1. VendorID
2. tpep_pickup_datetime
3. tpep_dropoff_datetime
4. passenger_count
5. trip_distance
6. RatecodeID
7. store_and_fwd_flag
8. PULocationID
9. DOLocationID
10. payment_type
11. fare_amount
12. extra
13. mta_tax
14. tip_amount
15. tolls_amount
16. improvement_surcharge
17. total_amount
18. congestion_surcharge
19. zone_PU
20. LocationID_PU
21. Borough_PU
22. zone_DO
23. LocationID_DO
24. Borough_DO
25. VendorID_1
26. VendorID_2
27. RateCode_1
28. RateCode_2
29. RateCode_3
30. RateCode_4
31. RateCode_5
32. RateCode_6
33. RateCode_99
34. payment_type_1
35. payment_type_2
36. payment_type_3
37. payment_type_4
38. payment_type_5
39. Bronx_PU
40. Brooklyn_PU
41. Manhattan_PU
42. Queens_PU
43. Staten_Island_PU
44. Bronx_DO
45. Brooklyn_DO
46. Manhattan_DO
47. Queens_DO
48. Staten_Island_DO


# Do your column names correspond to what those columns store? 

The column names appeaer to each describe what each column holds. Each column is described as: 
1. VendorID: A code indicating the TPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc. : The date and time when the meter was engaged.
2. tpep_pickup_datetime: The date and time when the meter was engaged.
3. tpep_dropoff_datetime: The date and time when the meter was disengaged.
4. passenger_count: The number of passengers in the vehicle. This is a driver-entered value.
5. trip_distance: The elapsed trip distance in miles reported by the taximeter.
6. RatecodeID:The final rate code in effect at the end of the trip. 1= Standard rate 2=JFK 3=Newark 4=Nassau or Westchester 5=Negotiated fare 6=Group ride
7. store_and_fwd_flag: This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server. Y= store and forward trip N= not a store and forward trip
8. PULocationID: TLC Taxi Zone in which the taximeter was engaged
9. DOLocationID: TLC Taxi Zone in which the taximeter was disengaged
10. payment_type: A numeric code signifying how the passenger paid for the trip. 1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided trip
11. fare_amount: The time-and-distance fare calculated by the meter.
12. extra: Miscellaneous extras and surcharges. Currently, this only includes the $0.50 and $1 rush hour and overnight charges.
13. mta_tax: 0.50 cents MTA tax that is automatically triggered based on the metered rate in use.
14. tip_amount: Tip amount – This field is automatically populated for credit card tips. Cash tips are not included.
15. tolls_amount: Total amount of all tolls paid in trip.
16. improvement_surcharge: 0.30 cents improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015.
17. total_amount: The total amount charged to passengers. Does not include cash tips.
18. congestion_surcharge: 
19. zone_PU: The area zone of where the pick up location was.
20. LocationID_PU: The taxi zone number of where the pick up location was.
21. Borough_PU: The borough of the pick up location.
22. zone_DO: The area zone of where the pick up location was.
23. LocationID_DO: The taxi zone number of where the pick up location was.
24. Borough_DO: The borough of the pick up location.

# 2.2 Data Types <a id='2.2_Data_Types'></a>

The next part is going to explore what the data types of each column. This will help gain a better understanding of what we may need to clean and how to clean it.

In [18]:
pandemic_taxi_data.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
zone_PU                   object
LocationID_PU              int64
Borough_PU                object
zone_DO                   object
LocationID_DO              int64
Borough_DO                object
VendorID_1                 uint8
VendorID_2                 uint8
RateCode_1                 uint8
RateCode_2                 uint8
RateCode_3                 uint8
RateCode_4                 uint8
RateCode_5

The .dtypes method shows and the datatypes of 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type',
'zone_PU', 'LocationID_PU', 'Borough_PU', 'zone_DO','LocationID_DO', and 'Borough_DO' are supposed to be categorical but are marked as in64 objects. The next cells will convert the categorical columns to a category object.

In [19]:
# Iterate through the columns and change them into category types
list_of_categorical = ['store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type',
                      'zone_PU', 'LocationID_PU', 'Borough_PU',
                      'zone_DO', 'LocationID_DO', 'Borough_DO']
for item in list_of_categorical:
    pandemic_taxi_data[item] = pandemic_taxi_data[item].astype('category')

In [20]:
pandemic_taxi_data.dtypes

VendorID                  float64
tpep_pickup_datetime       object
tpep_dropoff_datetime      object
passenger_count           float64
trip_distance             float64
RatecodeID                float64
store_and_fwd_flag       category
PULocationID             category
DOLocationID             category
payment_type             category
fare_amount               float64
extra                     float64
mta_tax                   float64
tip_amount                float64
tolls_amount              float64
improvement_surcharge     float64
total_amount              float64
congestion_surcharge      float64
zone_PU                  category
LocationID_PU            category
Borough_PU               category
zone_DO                  category
LocationID_DO            category
Borough_DO               category
VendorID_1                  uint8
VendorID_2                  uint8
RateCode_1                  uint8
RateCode_2                  uint8
RateCode_3                  uint8
RateCode_4    

The appropriate datatypes have been changed from int64s into categorical data types.

# 2.3 Description Of Columns <a id= '2.3_Descriptions_Of_Columns'></a>

The next cell will describe what the columns statistical values look like. It'll give us an idea of how the data is behaving.

# 2.4 Counts and Percents Unique Values <a id='2.4_Counts_And_Percents'></a>

The following cell will count the cardinality of each column and see how their ratios are.

In [21]:
for item in pandemic_taxi_data.columns:
    len_unique = len(pandemic_taxi_data[item].unique())
    print('The cardinality of {} is: {}'.format(item, len_unique))  

The cardinality of VendorID is: 3
The cardinality of tpep_pickup_datetime is: 6336458
The cardinality of tpep_dropoff_datetime is: 6342811
The cardinality of passenger_count is: 11
The cardinality of trip_distance is: 5544
The cardinality of RatecodeID is: 8
The cardinality of store_and_fwd_flag is: 3
The cardinality of PULocationID is: 258
The cardinality of DOLocationID is: 258
The cardinality of payment_type is: 6
The cardinality of fare_amount is: 8682
The cardinality of extra is: 78
The cardinality of mta_tax is: 16
The cardinality of tip_amount is: 3932
The cardinality of tolls_amount is: 1461
The cardinality of improvement_surcharge is: 3
The cardinality of total_amount is: 13972
The cardinality of congestion_surcharge is: 10
The cardinality of zone_PU is: 258
The cardinality of LocationID_PU is: 258
The cardinality of Borough_PU is: 6
The cardinality of zone_DO is: 258
The cardinality of LocationID_DO is: 258
The cardinality of Borough_DO is: 6
The cardinality of VendorID_1 is:

# 3. Data Cleaning <a id='3_Data_Cleaning'></a>

This section will cover how I clean the data's nan values, outliers and checking for duplicates.

In [23]:
pandemic_taxi_data.isna().sum()

VendorID                 278223
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          278223
trip_distance                 0
RatecodeID               278223
store_and_fwd_flag       278223
PULocationID                  0
DOLocationID                  0
payment_type             278223
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge          0
zone_PU                       0
LocationID_PU                 0
Borough_PU                 1475
zone_DO                       0
LocationID_DO                 0
Borough_DO                29108
VendorID_1                    0
VendorID_2                    0
RateCode_1                    0
RateCode_2                    0
RateCode_3                    0
RateCode_4                    0
RateCode_5                    0
RateCode

In [24]:
col_with_null_vals = ['VendorID', 'passenger_count', 'RatecodeID',
                      'payment_type', 'Manhattan_PU', 'Queens_PU', 'Bronx_PU', 'Brooklyn_PU',
                     'Staten_Island_PU', 'Manhattan_DO', 'Queens_DO', 'Bronx_DO', 'Brooklyn_DO',
                     'Staten_Island_DO']

df_nulls = pandemic_taxi_data[col_with_null_vals].isnull()
df_nulls.to_csv('./datasetsTFNullTable.csv')

In [25]:
df_nulls.head()

,VendorID,passenger_count,RatecodeID,payment_type,Manhattan_PU,Queens_PU,Bronx_PU,Brooklyn_PU,Staten_Island_PU,Manhattan_DO,Queens_DO,Bronx_DO,Brooklyn_DO,Staten_Island_DO
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [26]:
df_nulls.replace({False: 0, True: 1}, inplace=True)

In [27]:
col_names_list = []
for col in df_nulls.columns:
    col_names_list.append(col + '_isna')
df_nulls.columns = col_names_list

In [28]:
df_nulls.head()

,VendorID_isna,passenger_count_isna,RatecodeID_isna,payment_type_isna,Manhattan_PU_isna,Queens_PU_isna,Bronx_PU_isna,Brooklyn_PU_isna,Staten_Island_PU_isna,Manhattan_DO_isna,Queens_DO_isna,Bronx_DO_isna,Brooklyn_DO_isna,Staten_Island_DO_isna
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
pandemic_taxi_data = pandemic_taxi_data.join(df_nulls)
pandemic_taxi_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,zone_PU,LocationID_PU,Borough_PU,zone_DO,LocationID_DO,Borough_DO,VendorID_1,VendorID_2,RateCode_1,RateCode_2,RateCode_3,RateCode_4,RateCode_5,RateCode_6,RateCode_99,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,Bronx_PU,Brooklyn_PU,Manhattan_PU,Queens_PU,Staten_Island_PU,Bronx_DO,Brooklyn_DO,Manhattan_DO,Queens_DO,Staten_Island_DO,VendorID_isna,passenger_count_isna,RatecodeID_isna,payment_type_isna,Manhattan_PU_isna,Queens_PU_isna,Bronx_PU_isna,Brooklyn_PU_isna,Staten_Island_PU_isna,Manhattan_DO_isna,Queens_DO_isna,Bronx_DO_isna,Brooklyn_DO_isna,Staten_Island_DO_isna
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2.0,2020-01-01 00:24:22,2020-01-01 00:25:54,1.0,0.56,1.0,N,238,239,1.0,3.5,0.5,0.5,1.70,0.0,0.3,9.00,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2.0,2020-01-01 00:46:46,2020-01-01 00:49:17,1.0,0.62,1.0,N,238,239,1.0,4.5,0.5,0.5,1.00,0.0,0.3,9.30,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,2020-01-01 00:17:53,2020-01-01 00:22:47,1.0,0.80,1.0,Y,238,239,1.0,5.5,3.0,0.5,1.85,0.0,0.3,11.15,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2.0,2020-01-01 00:49:45,2020-01-01 00:54:12,1.0,0.86,1.0,N,238,239,1.0,5.5,0.5,0.5,1.86,0.0,0.3,11.16,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 3.1 Filling NULL values <a id='3.1_Filling_NULL_values'></a>

This lets us know that there are seven columns with NaN values. What I'll do with the NaN values is substitute the NaNs with 0s

In [30]:
pandemic_taxi_data.describe().T

,count,mean,std,min,25%,50%,75%,max
VendorID,16394995.0,1.664669e+00,0.472106,1.00,1.00,2.00,2.00,2.00
passenger_count,16394995.0,1.491998e+00,1.136433,0.00,1.00,1.00,2.00,9.00
trip_distance,16673218.0,3.073729e+00,165.676680,-30.62,0.98,1.62,3.00,297004.51
RatecodeID,16394995.0,1.043600e+00,0.676544,1.00,1.00,1.00,1.00,99.00
fare_amount,16673218.0,1.253740e+01,195.443756,-1259.00,6.50,9.00,14.00,671100.14
extra,16673218.0,1.133242e+00,122.456794,-27.00,0.00,0.50,2.50,500000.80
mta_tax,16673218.0,5.230166e-01,122.450446,-0.50,0.50,0.50,0.50,500000.50
tip_amount,16673218.0,2.132019e+00,2.600147,-493.22,0.00,1.92,2.86,1100.00
tolls_amount,16673218.0,3.209923e-01,1.635388,-40.00,0.00,0.00,0.00,911.75
improvement_surcharge,16673218.0,2.979066e-01,0.034736,-0.30,0.30,0.30,0.30,0.30


In [31]:
a = (pandemic_taxi_data['tpep_dropoff_datetime'].astype('datetime64') - 
 pandemic_taxi_data['tpep_pickup_datetime'].astype('datetime64'))

In [32]:
mins = [(a[i].components[1] * 60) + a[i].components[2] + (round(a[i].components[3] / 60, 2)) for i in range(len(a))]
mins

[4.8,
 1.53,
 2.52,
 4.9,
 4.45,
 5.67,
 7.22,
 5.22,
 3.27,
 5.8,
 5.98,
 7.65,
 3.83,
 1.52,
 5.55,
 7.05,
 5.17,
 10.13,
 5.07,
 0.92,
 4.78,
 3.65,
 5.73,
 5.05,
 2.85,
 7.72,
 4.78,
 3.45,
 3.7199999999999998,
 4.0,
 2.93,
 4.55,
 7.32,
 4.88,
 8.23,
 6.95,
 1.9,
 3.9,
 3.93,
 4.72,
 4.12,
 6.33,
 2.1,
 4.22,
 2.73,
 4.75,
 5.85,
 2.27,
 5.33,
 1.38,
 2.88,
 5.27,
 4.68,
 7.25,
 2.22,
 2.12,
 4.82,
 2.48,
 3.33,
 1.98,
 3.33,
 4.02,
 6.17,
 5.65,
 4.6,
 4.58,
 3.2800000000000002,
 4.27,
 2.9699999999999998,
 3.7199999999999998,
 5.55,
 3.85,
 4.28,
 7.55,
 7.6,
 5.97,
 8.7,
 3.95,
 4.25,
 4.78,
 8.82,
 7.55,
 1.8199999999999998,
 1.42,
 4.2,
 3.18,
 7.42,
 3.4,
 6.13,
 7.83,
 4.1,
 3.27,
 1.88,
 5.37,
 5.85,
 4.5,
 3.0,
 0.97,
 5.4,
 4.3,
 4.83,
 3.07,
 4.9,
 3.03,
 4.22,
 5.37,
 3.08,
 3.13,
 3.9699999999999998,
 1.13,
 3.58,
 2.38,
 4.9,
 3.7,
 4.3,
 2.42,
 3.08,
 5.88,
 2.5300000000000002,
 3.12,
 4.48,
 3.5,
 1.6,
 6.6,
 2.5,
 2.32,
 4.5,
 3.55,
 4.62,
 2.55,
 5.07,
 4.85,
 6.

In [33]:
pandemic_taxi_data['Mins_In_Ride'] = mins
pandemic_taxi_data.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,zone_PU,LocationID_PU,Borough_PU,zone_DO,LocationID_DO,Borough_DO,VendorID_1,VendorID_2,RateCode_1,RateCode_2,RateCode_3,RateCode_4,RateCode_5,RateCode_6,RateCode_99,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,Bronx_PU,Brooklyn_PU,Manhattan_PU,Queens_PU,Staten_Island_PU,Bronx_DO,Brooklyn_DO,Manhattan_DO,Queens_DO,Staten_Island_DO,VendorID_isna,passenger_count_isna,RatecodeID_isna,payment_type_isna,Manhattan_PU_isna,Queens_PU_isna,Bronx_PU_isna,Brooklyn_PU_isna,Staten_Island_PU_isna,Manhattan_DO_isna,Queens_DO_isna,Bronx_DO_isna,Brooklyn_DO_isna,Staten_Island_DO_isna,Mins_In_Ride
16673213,NaN,2020-02-25 14:10:00,2020-02-25 14:39:00,NaN,12.31,NaN,NaN,139,2,NaN,35.60,5.5,0.5,0.00,4.60,0.3,46.50,0.0,Laurelton,139,Queens,Jamaica Bay,2,Queens,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,29.00
16673214,NaN,2020-04-15 09:05:00,2020-04-15 09:43:00,NaN,13.48,NaN,NaN,139,2,NaN,36.77,0.0,0.5,2.75,2.29,0.3,42.61,0.0,Laurelton,139,Queens,Jamaica Bay,2,Queens,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,38.00
16673215,1.0,2020-02-19 16:27:23,2020-02-19 16:37:06,1.0,1.50,1.0,N,2,2,1.0,8.50,1.0,0.5,2.05,0.00,0.3,12.35,0.0,Jamaica Bay,2,Queens,Jamaica Bay,2,Queens,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9.72
16673216,2.0,2020-03-26 03:06:52,2020-03-26 03:08:55,3.0,0.14,1.0,N,199,199,4.0,-3.50,-0.5,-0.5,0.00,0.00,-0.3,-4.80,0.0,Rikers Island,199,Bronx,Rikers Island,199,Bronx,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.05
16673217,2.0,2020-03-26 03:06:52,2020-03-26 03:08:55,3.0,0.14,1.0,N,199,199,2.0,3.50,0.5,0.5,0.00,0.00,0.3,4.80,0.0,Rikers Island,199,Bronx,Rikers Island,199,Bronx,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.05


In [34]:
pandemic_taxi_data['VendorID'] = pandemic_taxi_data['VendorID'].fillna(0)
pandemic_taxi_data['passenger_count'] = pandemic_taxi_data['passenger_count'].fillna(0)
pandemic_taxi_data['RatecodeID'] = pandemic_taxi_data['RatecodeID'].fillna(0)

In [35]:
pandemic_taxi_data['store_and_fwd_flag'] = pandemic_taxi_data['store_and_fwd_flag'].astype('string')
pandemic_taxi_data['store_and_fwd_flag'] = pandemic_taxi_data['store_and_fwd_flag'].fillna('NK') # Not Known
pandemic_taxi_data['store_and_fwd_flag'] = pandemic_taxi_data['store_and_fwd_flag'].astype('category')

pandemic_taxi_data['payment_type'] = pandemic_taxi_data['payment_type'].fillna(5) # For Unknown

pandemic_taxi_data['Borough_PU'] = pandemic_taxi_data['Borough_PU'].astype('string')
pandemic_taxi_data['Borough_PU'] = pandemic_taxi_data['Borough_PU'].fillna('Unknown')
pandemic_taxi_data['Borough_PU'] = pandemic_taxi_data['Borough_PU'].astype('category')

pandemic_taxi_data['Borough_DO'] = pandemic_taxi_data['Borough_DO'].astype('string')
pandemic_taxi_data['Borough_DO'] = pandemic_taxi_data['Borough_DO'].fillna('Unknown')
pandemic_taxi_data['Borough_DO'] = pandemic_taxi_data['Borough_DO'].astype('category')

In [36]:
pandemic_taxi_data.isna().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
                        ..
Queens_DO_isna           0
Bronx_DO_isna            0
Brooklyn_DO_isna         0
Staten_Island_DO_isna    0
Mins_In_Ride             0
Length: 63, dtype: int64

# 3.2 Checking For Outliers <a id='3.2_Checking_For_Duplicates'></a>

Any duplicates based off VendorID, trip_distance, total_amount should be dropped

In [37]:
pandemic_taxi_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,zone_PU,LocationID_PU,Borough_PU,zone_DO,LocationID_DO,Borough_DO,VendorID_1,VendorID_2,RateCode_1,RateCode_2,RateCode_3,RateCode_4,RateCode_5,RateCode_6,RateCode_99,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,Bronx_PU,Brooklyn_PU,Manhattan_PU,Queens_PU,Staten_Island_PU,Bronx_DO,Brooklyn_DO,Manhattan_DO,Queens_DO,Staten_Island_DO,VendorID_isna,passenger_count_isna,RatecodeID_isna,payment_type_isna,Manhattan_PU_isna,Queens_PU_isna,Bronx_PU_isna,Brooklyn_PU_isna,Staten_Island_PU_isna,Manhattan_DO_isna,Queens_DO_isna,Bronx_DO_isna,Brooklyn_DO_isna,Staten_Island_DO_isna,Mins_In_Ride
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.80
1,2.0,2020-01-01 00:24:22,2020-01-01 00:25:54,1.0,0.56,1.0,N,238,239,1.0,3.5,0.5,0.5,1.70,0.0,0.3,9.00,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.53
2,2.0,2020-01-01 00:46:46,2020-01-01 00:49:17,1.0,0.62,1.0,N,238,239,1.0,4.5,0.5,0.5,1.00,0.0,0.3,9.30,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.52
3,1.0,2020-01-01 00:17:53,2020-01-01 00:22:47,1.0,0.80,1.0,Y,238,239,1.0,5.5,3.0,0.5,1.85,0.0,0.3,11.15,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.90
4,2.0,2020-01-01 00:49:45,2020-01-01 00:54:12,1.0,0.86,1.0,N,238,239,1.0,5.5,0.5,0.5,1.86,0.0,0.3,11.16,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.45


Time to remove wrongly inputted data

In [39]:
pandemic_taxi_data = pandemic_taxi_data.drop(['VendorID', 'RatecodeID', 'payment_type', 'PULocationID',
                                              'DOLocationID', 'LocationID_PU','LocationID_DO', 
                                              'Borough_PU', 'Borough_DO', 'zone_PU', 'zone_DO'], axis=1)
pandemic_taxi_data.shape

(16673218, 52)

# 3.3 Finished Cleaning <a id='3.3_Finished_Cleaning'></a>

In [42]:
pandemic_taxi_data.to_csv('datasets/Clean_Raw_Data.csv')